## Part 2. Time Series analysis of Master data file

In part 2 we want to explore our data and its trends over time. Hopefully we will uncover further features and behaviours of our sample.

In [11]:
import pandas as pd

df_master = pd.read_csv('master_file.csv', parse_dates=[0], infer_datetime_format=True)
df_master.head()

,Date,Type,Patient,Doctor,Town
0,2020-01-02,Home,patient_17268,doctor_47,town_1
1,2020-01-02,Clinic,patient_19126,doctor_47,town_1
2,2020-01-02,Home,patient_19521,doctor_47,town_1
3,2020-01-02,Clinic,patient_3399,doctor_47,town_1
4,2020-01-02,Clinic,patient_18881,doctor_47,town_1


Let's do a simple plot of the behaviour over the trail period. We will need to group the dates together to aggregate the other fields.

In [16]:
df_grouped_days = df_master.groupby(by='Date', as_index=False).count()
df_grouped_days.head()

,Date,Type,Patient,Doctor,Town
0,2020-01-02,494,494,494,494
1,2020-01-03,489,489,489,489
2,2020-01-04,500,500,500,500
3,2020-01-05,500,500,500,500
4,2020-01-06,494,494,494,494


Grouping the dates by counts shows us the total number of patient/doctor interactions in a given day. The 'Patient' column is the most intuitive column here as we can interpret it as "The total number of patients seen on this date".

*As an aside,  the values in the other columns are an exact match but are less intuitive to describe i.e. "The total number of visit types registered on a given date". This is the same as the number of "patients seen", but harder to convey to the end audience. This should be a primary consideration in data analysis/visualisation.*

In [24]:
import plotly.express as px

fig = px.line(df_grouped_days, x='Date', y='Patient')
fig.update_layout(yaxis_title='No. of Patients Seen', title='Total # of patients seen per day')

fig.show()

This is an interesting profile. The number of patients seen each day is hovering just under 500 everyday. However, every ~2 weeks or so theres a sudden jump in the count, almost reaching 750 in a single day, a single day increase of 50%! 

Perhaps there's a "Healthcare Drive" every two weeks that we are unaware of. To determine if this might be the case, we should restructure our Date column to include the days of the week. A fortnightly drive should be on the same day of the week for consistencya dn planning.

To figure this, we will filter our grouped data for days which have more than 700 visits, and then determine the day of the week that they occured.

In [51]:
df_super_day = df_grouped_days.loc[df_grouped_days['Patient']>700]

df_super_day['DOW'] = df_super_day.Date.dt.day_name()

df_super_day.head(10)

<ipython-input-51-5468f2595180>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Date,Type,Patient,Doctor,Town,DOW
14,2020-01-16,739,739,739,739,Thursday
29,2020-01-31,738,738,738,738,Friday
44,2020-02-15,743,743,743,743,Saturday
59,2020-03-01,740,740,740,740,Sunday
74,2020-03-16,735,735,735,735,Monday
89,2020-03-31,740,740,740,740,Tuesday
104,2020-04-15,737,737,737,737,Wednesday
119,2020-04-30,735,735,735,735,Thursday
134,2020-05-15,732,732,732,732,Friday
149,2020-05-30,731,731,731,731,Saturday


Now this is more suprising. The day of the week where these "super days" are occurring are every 15 days, incrementing to the next day of the week. 
This is highly bizzare if there was an event or drive associated with these numbers, and now causes the numbers themselves to be suspect.

We should investgate are these spikes caused by all towns equally, or a few outliers.



In [72]:
df_grouped_towns = df_master.groupby(by=['Date',], as_index=True).count()
df_grouped_towns.head()

,Type,Patient,Doctor,Town
Date,,,,
2020-01-02,494,494,494,494
2020-01-03,489,489,489,489
2020-01-04,500,500,500,500
2020-01-05,500,500,500,500
2020-01-06,494,494,494,494


Now let's focus on the 'super days'.

In [63]:
df_super_day['Date']

14    2020-01-16
29    2020-01-31
44    2020-02-15
59    2020-03-01
74    2020-03-16
89    2020-03-31
104   2020-04-15
119   2020-04-30
134   2020-05-15
149   2020-05-30
164   2020-06-14
179   2020-06-29
194   2020-07-14
209   2020-07-29
224   2020-08-13
239   2020-08-28
Name: Date, dtype: datetime64[ns]

In [85]:
df_master['Date']== df_super_day['Date']


ValueError: Can only compare identically-labeled Series objects